In [1]:
!pip install schedule


In [2]:
import pandas as pd
import spacy
import random
import os

In [3]:
from openpyxl import Workbook, load_workbook
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import schedule
import time
import smtplib
from email.mime.text import MIMEText
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [4]:
data = pd.read_csv('reduced_emotion_dataset.csv')  # Mood Dataset
affirmations = pd.read_csv('affirmation new.csv')  # Affirmation Dataset

# Step 3: Preprocess Text Data with spaCy
nlp = spacy.load('en_core_web_sm')


In [ ]:
def preprocess_text_spacy(text):
    # Convert text to lowercase and process with spaCy
    doc = nlp(text.lower())
    # Remove punctuation and stopwords, and tokenize
    tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(tokens)

data['Cleaned_Text'] = data['Text'].apply(preprocess_text_spacy)

In [ ]:
# Step 4: Feature Extraction
vectorizer = TfidfVectorizer(max_features=5000)  # Limit to 5000 features
X = vectorizer.fit_transform(data['Cleaned_Text']).toarray()
y = data['Mood']




In [ ]:
print(data['Mood'].value_counts())


In [ ]:
# Step 5: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

# Step 6: Train a Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Step 7: Evaluate the Model
y_pred = model.predict(X_test)
y_test = y_test.astype(str)
y_pred = y_pred.astype(str)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy Score:", (accuracy_score(y_test, y_pred))*100, "%")

In [ ]:
from sklearn.tree import DecisionTreeClassifier


In [ ]:
# train model with DecisionTreeClassifier
df_model = DecisionTreeClassifier(random_state=42)
df_model.fit(X_train, y_train)


In [ ]:
dt_pred  = df_model.predict(X_test)
y_test = y_test.astype(str)
dt_pred = dt_pred.astype(str)
print("Confusion Matrix:\n", confusion_matrix(y_test, dt_pred))
print("Classification Report:\n", classification_report(y_test, dt_pred))
print("Accuracy Score:", (accuracy_score(y_test, dt_pred))*100, "%")

In [ ]:
# Step 8: Analyze the Mood from User Input
def analyze_mood(user_input):
    cleaned_input = preprocess_text_spacy(user_input)
    input_vector = vectorizer.transform([cleaned_input]).toarray()
    mood_probabilities = model.predict_proba(input_vector)[0]
    mood_classes = model.classes_
    mood_analysis = {mood_classes[i]: mood_probabilities[i] for i in range(len(mood_classes))}
    return mood_analysis

In [ ]:
# Step 9: Generate Affirmations
def generate_affirmation(mood_analysis):
    predicted_mood = max(mood_analysis, key=mood_analysis.get)
    filtered_affirmations = affirmations[affirmations['Mood'] == predicted_mood]['Affirmation'].tolist()
    return random.choice(filtered_affirmations)

In [ ]:
# Step 10: Save User Email and Affirmation
def save_email_and_affirmation(email, affirmation):
    file_path = 'user_affirmations.xlsx'

    # Create a new workbook if the file doesn't exist
    if not os.path.exists(file_path):
        workbook = Workbook()
        sheet = workbook.active
        sheet.append(["Email", "Affirmation"])
        workbook.save(file_path)

    # Add the email and affirmation to the file
    workbook = load_workbook(file_path)
    sheet = workbook.active
    sheet.append([email, affirmation])
    workbook.save(file_path)
    print(f"Your affirmation has been saved and will be sent to {email} tomorrow morning.")

In [ ]:
# Step 11: Send Affirmation Email
def send_email(email, affirmation):
    sender_email = "youremail@gmail.com"  # Replace with your email address
    sender_password = "#16digitapppw"  # Replace with your Gmail app password
    subject = "Your Morning Affirmation 🌞"
    body = f"Good morning!\n\nHere’s an affirmation for you today:\n\n{affirmation}\n\nHave a wonderful day!"

    msg = MIMEText(body, "plain", "utf-8")
    msg['Subject'] = subject
    msg['From'] = sender_email
    msg['To'] = email

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, email, msg.as_string())
            print(f"Affirmation sent to {email}")
    except Exception as e:
        print("Error sending email:", e)

In [ ]:
 #Step 12: Schedule Email Sending
def send_scheduled_emails():
    file_path = 'user_affirmations.xlsx'

    if not os.path.exists(file_path):
            print("No emails to send.")
            return

# Create a new Excel file with headers
    workbook = Workbook()
    sheet = workbook.active
    sheet.append(["Email", "Affirmation"])  # Add column headers
    workbook.save(file_path)
    st.write("Created a new user_affirmations.xlsx file.")

    for row in sheet.iter_rows(min_row=2, values_only=True):
        email, affirmation = row
        send_email(email, affirmation)

    # Clear the file after sending all emails
    sheet.delete_rows(2, sheet.max_row)
    workbook.save(file_path)

# Schedule the email sending at 7 AM every day
schedule.every().day.at("10:53").do(send_scheduled_emails)

In [ ]:
# Step 13: Interactive Input-Output

while True:
    user_input = input("How are you feeling today? Describe your mood: ")
    email = input("Please enter your email address: ")

    mood_analysis = analyze_mood(user_input)
    print("\nMood Analysis:")
    for mood, prob in mood_analysis.items():
        print(f"{mood}: {prob*100:.2f}%")

    # Analyze mood and generate affirmation
    predicted_mood = analyze_mood(user_input)
    affirmation = generate_affirmation(predicted_mood)

    # Display affirmation immediately
    print("\nHere's an affirmation for you:")
    print(affirmation)

    # Save email and affirmation for later sending
    save_email_and_affirmation(email, affirmation)

    # Check if the user wants to continue
    cont = input("Do you want to input another email? (yes/no): ").lower()
    if cont != 'yes':
        break

# Run the scheduler (in a long-running process)
while True:
    schedule.run_pending()
    time.sleep(60)